<a href="https://colab.research.google.com/github/elgrhy/objects/blob/main/TransformerModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install torch
! pip install pandas
! pip install -U scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 103.9 MB/s eta 0:00:00


In [4]:
from google.colab import files
uploaded = files.upload()

Saving video_data (5).csv to video_data (5).csv


In [29]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import OneHotEncoder
from transformers import BertTokenizer, BertModel, BertForSequenceClassification

# device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Load data
df = pd.read_csv('video_data (5).csv')

# Encoder
encoder = OneHotEncoder(sparse=False)
train_labels_one_hot = encoder.fit_transform(df['likes'].values.reshape(-1, 1))
num_labels = len(encoder.categories_[0])

# Tokenize text using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_input_ids = []
for sent in df['hashtags']:
  encoded_sent = tokenizer.encode(sent, add_special_tokens=True, truncation=True, max_length=512)
  train_input_ids.append(encoded_sent)

# Split data into training and validation sets
train_texts, val_texts, train_labels, val_labels = train_test_split(df['hashtags'].tolist(), train_labels_one_hot, test_size=0.2)

# Convert data to tensors and move to GPU (if available)
num_classes = train_labels_one_hot.shape[1]
train_tokens = tokenizer.batch_encode_plus(train_texts, padding=True, truncation=True, return_tensors='pt')
train_input_ids = train_tokens['input_ids'].to(device)
train_attention_mask = train_tokens['attention_mask'].to(device)
train_labels = torch.tensor(train_labels, dtype=torch.float32).view(-1, num_classes).to(device)

val_tokens = tokenizer.batch_encode_plus(val_texts, padding=True, truncation=True, return_tensors='pt')
val_input_ids = val_tokens['input_ids'].to(device)
val_attention_mask = val_tokens['attention_mask'].to(device)
val_labels = torch.tensor(val_labels, dtype=torch.float32).view(-1, num_classes).to(device)

# Define model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes, output_hidden_states=True)
model.to(device)

# Train only the classifier layer
for name, param in model.named_parameters():
    if 'classifier' not in name:
        param.requires_grad = False

# Set up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 3
num_training_steps = len(train_input_ids) * num_epochs
lr_scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, 2, T_mult=2, eta_min=1e-6)

batch_size = 16  # set the batch size
num_batches = len(train_input_ids) // batch_size  # calculate the number of batches

pos_weight = torch.ones(num_classes, dtype=torch.float32)

# Define loss function
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
criterion.to(device)

# Train model
for epoch in range(num_epochs):
    model.train()
    for i in range(num_batches):
        start_idx = i * batch_size
        end_idx = (i + 1) * batch_size

    # Get batch inputs and labels
    batch_input_ids = train_input_ids[start_idx:end_idx]
    batch_attention_mask = train_attention_mask[start_idx:end_idx]
    batch_labels = train_labels[start_idx:end_idx].view(-1, num_classes).to(device)

    # Move batch inputs and labels to device
    batch_input_ids = batch_input_ids.to(device)
    batch_attention_mask = batch_attention_mask.to(device)
    batch_token_type_ids = batch_token_type_ids.to(device)
    batch_labels = batch_labels.to(device)

    # Get batch token type IDs
    batch_token_type_ids = train_tokens['token_type_ids'][start_idx:end_idx].to(device)

    #Zero gradients, perform a forward pass, compute loss, perform backward pass, and update parameters
    optimizer.zero_grad() # set gradients to zero
    output = model(batch_input_ids, batch_attention_mask, batch_token_type_ids) # forward pass
    loss = criterion(output.logits, batch_labels) # compute loss
    loss.backward() # backward pass to compute gradients
    optimizer.step() # update parameters based on gradients
    
    #Print progress every 100 batches
    if (i + 1) % 100 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Batch [{i + 1}/{num_batches}], Loss: {loss.item():.4f}")
        
# Evaluate model on validation data after each epoch
with torch.no_grad():
    val_output = model(val_input_ids, val_attention_mask, val_tokens['token_type_ids'].to(device))
    val_loss = criterion(val_output.logits, val_labels)
    val_pred_prob = nn.Softmax(dim=1)(val_output.logits)[:, 1] # take probability of positive class
    val_labels_2d = val_labels.cpu().numpy().reshape(-1, 1)
    val_pred_prob_2d = val_pred_prob.cpu().numpy().reshape(-1, 1)

    val_auc = roc_auc_score(val_labels_2d, val_pred_prob_2d)

print(f"Epoch [{epoch + 1}/{num_epochs}], Validation Loss: {val_loss.item():.4f}, Validation AUC: {val_auc:.4f}")

#Save the model
torch.save(model.state_dict(), 'bert_model.pth')


cuda


/usr/local/lib/python3.9/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassific

Epoch [1/3], Batch [500/500], Loss: 0.7110
Epoch [2/3], Batch [500/500], Loss: 0.7012
Epoch [3/3], Batch [500/500], Loss: 0.6925


ValueError: ignored